In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm.notebook import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("./나무문화/2022/나무문화_본문_2022_8월.csv", encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [3]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [4]:
df

,url,본문,org_idx,cleaning
0,https://blog.naver.com/smileseosin/222863511415,50m\n© NAVER Corp.\n송천도서관\n전라북도 전주시 덕진구 솔내로 21...,0,50m NAVER Corp 송천도서관 전라북도 전주시 덕진구 솔내로 212 송천도서...
1,https://blog.naver.com/bluex/222863510213,즐거운 책 놀이터\n독서교육 강사 이숙영입니다.\n어느 새 무더운 여름이 지나고\n...,1,즐거운 책 놀이터 독서교육 강사 이숙영입니다 어느 새 무더운 여름이 지나고 9월입니...
2,https://blog.naver.com/cmdsc809/222863508877,"다시 짧은 시간이 지났고,\n바람이 불었고,\n그렇게 앉아있는 동안\n나는 내가 이...",2,다시 짧은 시간이 지났고 바람이 불었고 그렇게 앉아있는 동안 나는 내가 이 순간을 ...
3,https://blog.naver.com/san1917/222863508568,멜크수도원\n멜크수도원\n2014년 4월 22일~2014년 5월2일\n베네룩스 3국...,3,멜크수도원 멜크수도원 2014년 4월 22일 2014년 5월2일 베네룩스 3국과 중...
4,https://blog.naver.com/hnmh2332/222863508119,추석연휴 놀러 갈만한곳 강원도 여행 평창 청옥산 육백마지기\n글&사진 : 초록바다 ...,4,추석연휴 놀러 갈만한곳 강원도 여행 평창 청옥산 육백마지기 글 사진 초록바다 최양훈...
...,...,...,...,...
20809,https://blog.naver.com/show656/222835904038,안녕하세요.\n쌍대킴입니다.\n오늘은 여름휴가를 맞이하여 찐 맛집 블로거 모드로 변...,20809,안녕하세요 쌍대킴입니다 오늘은 여름휴가를 맞이하여 찐 맛집 블로거 모드로 변신하여 ...
20810,https://blog.naver.com/sonia30/222835904165,강릉 여행::오죽헌 (feat.시립박물관)\n강릉에 왔다면 꼭 들려야 한다는 오죽헌...,20810,강릉 여행 오죽헌 feat 시립박물관 강릉에 왔다면 꼭 들려야 한다는 오죽헌 강릉 ...
20811,https://blog.naver.com/october2nd_/222835901895,주소\n대전 유성구 문화원로 41\n영업시간\n매주 월요일 휴무\n비가 많이 내린 ...,20811,주소 대전 유성구 문화원로 41 영업시간 매주 월요일 휴무 비가 많이 내린 일요일 ...
20812,https://blog.naver.com/moviemagazine/222835583839,자세한 줄거리\n결말포함 스포주의\n본 내용은 ‘감독판’을 기반하여 작성되었음을 알...,20812,자세한 줄거리 결말포함 스포주의 본 내용은 감독판 을 기반하여 작성되었음을 알려드립...


In [5]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

  0%|          | 0/20814 [00:00<?, ?it/s]

,url,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/smileseosin/222863511415,50m\n© NAVER Corp.\n송천도서관\n전라북도 전주시 덕진구 솔내로 21...,0,50m NAVER Corp 송천도서관 전라북도 전주시 덕진구 솔내로 212 송천도서...,[(50m NAVER Corp 송천도서관 전라북도 전주시 덕진구 솔내로 212 송천...
1,https://blog.naver.com/bluex/222863510213,즐거운 책 놀이터\n독서교육 강사 이숙영입니다.\n어느 새 무더운 여름이 지나고\n...,1,즐거운 책 놀이터 독서교육 강사 이숙영입니다 어느 새 무더운 여름이 지나고 9월입니...,"[(즐거운 책 놀이터 독서교육 강사 이숙영입니다, 0, 24, None, []), ..."
2,https://blog.naver.com/cmdsc809/222863508877,"다시 짧은 시간이 지났고,\n바람이 불었고,\n그렇게 앉아있는 동안\n나는 내가 이...",2,다시 짧은 시간이 지났고 바람이 불었고 그렇게 앉아있는 동안 나는 내가 이 순간을 ...,[(다시 짧은 시간이 지났고 바람이 불었고 그렇게 앉아있는 동안 나는 내가 이 순간...
3,https://blog.naver.com/san1917/222863508568,멜크수도원\n멜크수도원\n2014년 4월 22일~2014년 5월2일\n베네룩스 3국...,3,멜크수도원 멜크수도원 2014년 4월 22일 2014년 5월2일 베네룩스 3국과 중...,[(멜크수도원 멜크수도원 2014년 4월 22일 2014년 5월2일 베네룩스 3국과...
4,https://blog.naver.com/hnmh2332/222863508119,추석연휴 놀러 갈만한곳 강원도 여행 평창 청옥산 육백마지기\n글&사진 : 초록바다 ...,4,추석연휴 놀러 갈만한곳 강원도 여행 평창 청옥산 육백마지기 글 사진 초록바다 최양훈...,[(추석연휴 놀러 갈만한곳 강원도 여행 평창 청옥산 육백마지기 글 사진 초록바다 최...
...,...,...,...,...,...
20809,https://blog.naver.com/show656/222835904038,안녕하세요.\n쌍대킴입니다.\n오늘은 여름휴가를 맞이하여 찐 맛집 블로거 모드로 변...,20809,안녕하세요 쌍대킴입니다 오늘은 여름휴가를 맞이하여 찐 맛집 블로거 모드로 변신하여 ...,"[(안녕하세요 쌍대킴입니다, 0, 12, None, []), (오늘은 여름휴가를 맞..."
20810,https://blog.naver.com/sonia30/222835904165,강릉 여행::오죽헌 (feat.시립박물관)\n강릉에 왔다면 꼭 들려야 한다는 오죽헌...,20810,강릉 여행 오죽헌 feat 시립박물관 강릉에 왔다면 꼭 들려야 한다는 오죽헌 강릉 ...,[(강릉 여행 오죽헌 feat 시립박물관 강릉에 왔다면 꼭 들려야 한다는 오죽헌 강...
20811,https://blog.naver.com/october2nd_/222835901895,주소\n대전 유성구 문화원로 41\n영업시간\n매주 월요일 휴무\n비가 많이 내린 ...,20811,주소 대전 유성구 문화원로 41 영업시간 매주 월요일 휴무 비가 많이 내린 일요일 ...,[(주소 대전 유성구 문화원로 41 영업시간 매주 월요일 휴무 비가 많이 내린 일요...
20812,https://blog.naver.com/moviemagazine/222835583839,자세한 줄거리\n결말포함 스포주의\n본 내용은 ‘감독판’을 기반하여 작성되었음을 알...,20812,자세한 줄거리 결말포함 스포주의 본 내용은 감독판 을 기반하여 작성되었음을 알려드립...,[(자세한 줄거리 결말포함 스포주의 본 내용은 감독판 을 기반하여 작성되었음을 알려...


In [6]:
# 데이터프레임을 1000개 단위로 나누기
chunk_size = 10
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129
df_130
df_131
df_132
df_133
df_134
df_135
df_136
df_137
df_138
df_139
df_140
df_141
df_142
df_143
df_144
df_145
df_146
df_147
df_148
df_149
df_150
df_151
df_152
df_153
df_154
df_155
df_156
df_157
df_158
df

In [2]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("./나무문화/2022/나무문화_본문_8월_전처리.csv", index=False, encoding='UTF-8')

In [7]:
df2 = pd.read_csv("./나무문화/2022/나무문화_본문_8월_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx


In [8]:
dfs['df_1']['split_list'].index

RangeIndex(start=0, stop=10, step=1)

In [9]:
len(dfs)

2082

In [10]:
for a in range(1,len(dfs)+1):
    for i in tqdm(dfs['df_%s'%a]['split_list'].index):
        for j in range(len(dfs['df_%s'%a]['split_list'][i])):
            split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
            print(split_str['org_idx'])
            split_df= pd.DataFrame(split_str)
            df2 = pd.concat([df2,split_df])
            df2.reset_index(drop=True, inplace=True)
            display.clear_output(wait=True) # 출력지우기
        else:
            time.sleep(1)

df2.reset_index(drop=True, inplace=True)
df2.to_csv("./나무문화/2022/나무문화_본문_8월_전처리.csv", index=False, encoding='UTF-8')
df2

[6535]


In [ ]:
df2.to_csv("./나무문화/2022/나무문화_본문_8월_전처리.csv", index=False, encoding='UTF-8')